Run experiments to compare quadratic underestimators with other type of estimators.


In [1]:
import pandas as pd
import numpy as np
import interval as ival
import uvarprob as uvpr
import bnb as bnb
import sub as sub
import ivalprocessor as ivproc
import pslprocessor as pslproc
import psqeprocessor as psqproc
import gridsearch as gs
from sortedcontainers import SortedKeyList
from collections import namedtuple

Define an array of test files to process

In [2]:
test_files = ["/tmp/shek.csv"]

Define some tuning parameters

In [3]:
# Maximal number of steps
max_steps = 1e6
# Tolerance
epsilon = 1e-2
# If True - global Lipschitz constant is used
global_lipschitz_interval = True
# If True - the record value is taken from the test database
known_record = False

In [4]:
def get_initial_recval(prob):
    if known_record:
        return prob.min_f
    else:
        return float('inf')

In [5]:
TestResult = namedtuple('TestResult', ['nsteps', 'record_value'])

In [6]:
def psl(prob, sym):
    psp = pslproc.PSLProcessor(rec_v = get_initial_recval(prob), rec_x = None, problem = prob, eps = epsilon, global_lipint = global_lipschitz_interval, use_symm_lipint = sym)
    sl = SortedKeyList(key = lambda s : s.level)
    subp = sub.Sub(0, 0, pslproc.PSLData(ival.Interval([prob.a, prob.b]),0))
    psp.compute_bounds(subp)
    sl.add(subp)
    cnt = max_steps
    steps = bnb.bnb(sl, cnt, psp)
#     print("steps performed: ", steps)
#     print("Record value = ", psp.rec_v, " at ", psp.rec_x);
#     print(sl)
    return TestResult(nsteps = steps, record_value = psp.rec_v)

In [7]:
def psqe(prob, sym):
    psp = psqproc.PSQEProcessor(rec_v = get_initial_recval(prob), rec_x = None, problem = prob, eps = epsilon, global_lipint = global_lipschitz_interval, use_symm_lipint = sym)
    sl = SortedKeyList(key = lambda s : s.level)
    subp = sub.Sub(0, 0, psqproc.PSQEData(ival.Interval([prob.a, prob.b]),0))
    psp.compute_bounds(subp)
    sl.add(subp)
    cnt = max_steps
    steps = bnb.bnb(sl, cnt, psp)
#     print("steps performed: ", steps)
#     print("Record value = ", psp.rec_v, " at ", psp.rec_x);
#     print(sl)
    return TestResult(nsteps = steps, record_value = psp.rec_v)

In [8]:
def psl_non_sym(prob):
    return psl(prob, False)

In [9]:
def psl_sym(prob):
    return psl(prob, True)

In [10]:
def psqe_non_sym(prob):
    return psqe(prob, False)

In [11]:
def psqe_sym(prob):
    return psqe(prob, True)

Tested methods

In [12]:
test_methods = {'LLI' : psl_non_sym, 'LLC' : psl_sym, 'QLI' : psqe_non_sym, 'QLC' : psqe_sym}

In [13]:
# test_methods = {'LLI' : psl_non_sym, 'QLI' : psqe_non_sym, 'QLC' : psqe_sym}

In [14]:
def run_method(method, prob):
    result = method(prob)
    return result

In [15]:
def run_test(test):
    prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x)
    print(prob.min_f)
    runs = {}
    for name, method in test_methods.items():
        result = run_method(method, prob)
#         print(name, "result :", result)
        runs[name] = result
    return runs



In [20]:
def run_tests(df):
    data = {}
    for row in df.itertuples():
        for name, tres in run_test(row).items():
            steps_key = name + '_steps'
            if data.get(steps_key) == None:
                data[steps_key] = [tres.nsteps]
            else:
                data[steps_key].append(tres.nsteps)
            record_key = name + '_record'
            if data.get(record_key) == None:
                data[record_key] = [tres.record_value]
            else:
                data[record_key].append(tres.record_value)
    print(data)
    perf_df = pd.DataFrame(data, index = df.index)
    print(perf_df)
    print(perf_df.diff(axis = 1, periods = 2))
    return perf_df

In [21]:
def read_problems(fname):
    data = pd.read_csv(fname, index_col = 'name', comment = '#')
    return data

Main testing scenario

In [27]:
for file in test_files:
    df = run_tests(read_problems(file))
    mean_data = df.mean()
#     print(mean_data['LLI_steps'])
    print("=============")
    steps_data = pd.Series(mean_data, index = ['LLC_steps', 'LLI_steps', 'QLC_steps', 'QLI_steps'])
    print(steps_data)

0.53993
1.4095
0.99992
0.41883
0.72143
0.4338
0.58908
0.51048
0.6859
0.42419
{'LLI_steps': [551839, 309368, 402303, 675551, 439164, 572815, 438548, 914419, 613046, 705365], 'LLI_record': [0.5399254341946542, 1.409502007312278, 0.9999221948575914, 0.41882713041733, 0.7214302935355311, 0.4338045561979252, 0.589076411889616, 0.5104762148331544, 0.6858992800482617, 0.42418803376133835], 'LLC_steps': [754797, 295817, 448523, 818613, 590765, 579841, 809869, 1000001, 819837, 708655], 'LLC_record': [0.5399254341946222, 1.409502007312265, 0.9999221948582324, 0.41882713041731234, 0.7214302935355152, 0.4338045561979053, 0.5890764118896218, 0.5104762148331512, 0.685899280048319, 0.4241880337614379], 'QLI_steps': [18426, 10822, 14651, 14699, 16703, 12087, 20000, 20194, 19887, 18728], 'QLI_record': [0.5399254345971413, 1.409502007335007, 0.999922196475712, 0.41882713079170425, 0.7214302935606903, 0.43380455673724666, 0.5890764119309315, 0.5104762152136937, 0.6858992801316037, 0.4241880343962273], 'Q

In [19]:
print(perf_df)

NameError: name 'perf_df' is not defined